In [1]:
!pip install requests

In [1]:
import requests
import json
from IPython.display import Image
import os


In [7]:
api_key='rRtAMsJ6LJjcXv4z5JKCs1Q7IeQ3tdXTb3bFrBw8'
url=f'https://api.nasa.gov/planetary/apod?api_key={api_key}'
response = requests.get(url)
data = response.json()
data

{'copyright': 'Clear Skies',
 'date': '2025-07-11',
 'explanation': "Transfusing sunlight as the sky grew darker, this exceptional display of noctilucent clouds was captured on July 10, reflected in the calm waters of Vallentuna Lake near Stockholm, Sweden. From the edge of space, about 80 kilometers above Earth's surface, the icy clouds themselves still reflect sunlight, even though the Sun is below the horizon as seen from the ground. Usually spotted at high latitudes in summer months, the night shining clouds have made a strong showing so far during the short northern summer nights. Also known as polar mesopheric clouds they are understood to form as water vapor driven into the cold upper atmosphere condenses on the fine dust particles supplied by disintegrating meteors or volcanic ash.",
 'hdurl': 'https://apod.nasa.gov/apod/image/2507/NLCreflectionsHeden.jpg',
 'media_type': 'image',
 'service_version': 'v1',
 'title': 'The Veins of Heaven',
 'url': 'https://apod.nasa.gov/apod/ima

In [8]:
image_url = data['hdurl']
Image(url=image_url,width=1000)

In [9]:
apod_date = data['date']
apod_date

'2025-07-11'

In [5]:
file_extension = os.path.splitext(image_url)[1]
filename = f"apod_{apod_date}{file_extension}" # Image will be saved in the current directory

# Download and save the image
image_response = requests.get(image_url, stream=True)
with open(filename, 'wb') as f: # Saves directly to the current directory
    for chunk in image_response.iter_content(1024):
        f.write(chunk)

print(f"Image saved to: {os.getcwd()}/{filename}")

Image saved to: c:\Users\kkapil\Documents\mlops-python\16-astro\dags/apod_2025-07-11.jpg
